In [42]:
from bs4 import BeautifulSoup
import requests
import json
import re
import pandas as pd
import math
from unidecode import unidecode

In [43]:
def pegar_html(link):  
    req=requests.get(link)
    html=BeautifulSoup(req.text,"html.parser")
    return html

url='https://www.drogariaveracruz.com.br/medicamentos/'


def num_pag(html):
    string=html.find_all('div',{'class':'text-center pt-3'})[1].text
    lista=string.split()
    paginas= int(lista[-1])
    return paginas

def get_name(html):
    NOMES=[]
    nome=html.find_all('h2',{'class':'title'})

    for elem in nome:
        elem=elem.text
        elem=elem.replace('\n                            ',"")
        elem=elem.replace('\n                        ',"")
        NOMES.append(elem)

    return NOMES


def get_sale_price(html):
    SALE_PRICE=[]
    sale_price=html.find_all('p',{'class':'sale-price'})

    for preço in sale_price:
        preço=preço.text
        preço=preço.replace('R$ ',"")
        preço=preço.replace('.','')
        preço=float(preço.replace(",","."))
        SALE_PRICE.append(preço)
    return SALE_PRICE


def get_full_price(html):
    FULL_PRICE=[]
    div=html.find_all('div',{'class':'prices'})
    for caixa in div:
        full_price=caixa.find('p',{'class':'unit-price'})
        if full_price==None:
            full_price=caixa.find('p',{'class':'sale-price'})
        full_price=full_price.text
        
        full_price=full_price.replace('\n                                            ',"")
        full_price=full_price.replace('\n                                        ',"")
        full_price=full_price.replace('R$ ','')
        full_price=full_price.replace('.',"")
        full_price=float(full_price.replace(',','.'))


        FULL_PRICE.append(full_price)
    return FULL_PRICE



def get_links_produtos(html):
    div=html.find_all('div',{'class':'item-product'})
    LINKS=[]
    for caixa  in div:
        link_tag=caixa.find('a',{'class':'item-image position-relative'})
        link=link_tag.get('href')
        link="https://www.drogariaveracruz.com.br/"+link
        LINKS.append(link)
    return LINKS



def get_ean(url):

    html=pegar_html(url)

    div=html.find('div',{'class':'d-flex flex-wrap text-muted mb-3'})
    
    tag_gtin=div.find('meta',{'itemprop':'gtin13'})
    gtin=tag_gtin.get('content')
    gtin=int(gtin)
    return gtin




def get_marca(url):

    html=pegar_html(url)

    div=html.find('div',{'class':'d-flex flex-wrap text-muted mb-3'})

    tag_marca=div.find('meta',{'itemprop':'brand'})
    marca=tag_marca.get('content')
    return marca

def desconto(S,C):
    DES=[]
    for i in range(len(S)):
        des=S[i]-C[i]
        if des <0:
            des=(-1)*des
        DES.append(des)
    return DES





In [44]:
    
url='https://www.drogariaveracruz.com.br/medicamentos/'
html=pegar_html(url)

numero_paginas=num_pag(html)

colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
df=pd.DataFrame(columns=colunas)


for pagina in range (numero_paginas):

    NOMES_PRODUTOS=[]
    PREÇO_C_DES=[]
    PREÇO_S_DES=[]
    LINKS_PRODUTOS=[]
    url=f'https://www.drogariaveracruz.com.br/medicamentos/?p={pagina}'
    html=pegar_html(url)

    NOMES_PRODUTOS=get_name(html)
    PREÇO_C_DES=get_sale_price(html)
    PREÇO_S_DES=get_full_price(html)

    LINKS_PRODUTOS=get_links_produtos(html)


    
    DESC=[]
    DESC=desconto(PREÇO_C_DES,PREÇO_S_DES)

    EAN=[]
    MARCA=[]
    for link in LINKS_PRODUTOS:
        ean=get_ean(link)
        marca=get_marca(link)

        EAN.append(ean)
        MARCA.append(marca)
    
    
    colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
    df1=pd.DataFrame(columns=colunas)

    df1["EAN"]=EAN
    df1["Marca"]=MARCA
    df1["Nome"]=NOMES_PRODUTOS
    df1["Preço com desconto"]=PREÇO_C_DES
    df1["Preço sem desconto"]=PREÇO_S_DES
    df1["Desconto"]= DESC


    df=pd.concat([df,df1])
df=df.reset_index(inplace=True,drop=True)

C:\Users\monon\AppData\Local\Temp\ipykernel_20168\3045521956.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df=pd.concat([df,df1])


KeyboardInterrupt: 